In [1]:
import numpy as np
from scipy.signal import welch
from muselsl import stream, list_muses, view

In [3]:
from pylsl import StreamInlet, resolve_byprop
from scipy.signal import welch, butter, lfilter

In [4]:
import numpy as np
from pylsl import StreamInlet, resolve_byprop

# First, resolve an EEG stream on the lab network
print("Looking for an EEG stream...")
streams = resolve_byprop('type', 'EEG', timeout=2)

if streams:
    # Create a new inlet to read from the stream
    inlet = StreamInlet(streams[0])
    
    while True:
        # Get a new sample from the inlet
        sample, timestamp = inlet.pull_sample(timeout=1.0)
        if sample:
            # Process the sample (e.g., append it to a buffer)
            print(sample)
else:
    print("No streams found")

Looking for an EEG stream...
No streams found


We will be using the butterwoth filter

In [ ]:
# Function to design a Butterworth bandpass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Function to apply a bandpass filter to the data
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Function to calculate average power in the beta band
def beta_band_power(data, fs):
    freqs, psd = welch(data, fs=fs, nperseg=len(data))
    beta_low, beta_high = 13, 30  # Beta band range in Hz
    idx_beta = np.logical_and(freqs >= beta_low, freqs <= beta_high)
    beta_power = np.mean(psd[idx_beta])
    return beta_power

# Define the sampling frequency (fs) based on your device's specifications
fs = 256  # Hz

# Define a threshold for detecting high concentration from beta power
# This should be calibrated based on the user's baseline beta power levels
high_concentration_threshold = 0.5  # This is an example value

# First, resolve an EEG stream on the lab network
print("Looking for an EEG stream...")
streams = resolve_byprop('type', 'EEG', timeout=2)

In [ ]:
if streams:
    # Create a new inlet to read from the stream
    inlet = StreamInlet(streams[0])
    
    # Initialize a buffer to store a certain number of samples
    # For example, to store 5 seconds of data at 256 Hz
    buffer_length = 5 * fs
    eeg_buffer = np.zeros((buffer_length, len(streams[0].channel_format)))
    
    print("Stream found and connected")

# Define beta band range
beta_low, beta_high = 13, 30  # Beta band range in Hz

while True:
    # Get a new sample from the inlet
    sample, timestamp = inlet.pull_sample(timeout=1.0)
    if sample:
        # Process the sample (e.g., append it to a buffer)
        # Shift the existing buffer and append new sample at the end
        eeg_buffer = np.roll(eeg_buffer, -1, axis=0)
        eeg_buffer[-1, :] = np.array(sample)
        
        # Process the buffer only when it's full
        if np.all(eeg_buffer[:, 0] != 0):
            # Preprocess the buffer - apply bandpass filter
            filtered_buffer = bandpass_filter(eeg_buffer, beta_low, beta_high, fs)
            
            # Calculate beta band power
            beta_power = beta_band_power(filtered_buffer, fs)
            
            # Check if beta power exceeds the threshold
            if beta_power > high_concentration_threshold:
                print("High concentration detected. Triggering game action.")
                # Here you would trigger a game action (e.g., select or move a piece)
                # trigger_game_action('move')  # Replace with your game action function

        else:
            print("No new samples found")